In [1]:
from os import environ as ENV
from time import time as now
import json
from tqdm.notebook import tqdm


In [2]:
import sys
sys.path.append('..')

from circuit_postprocess import *
from should_be_stdlib import *
from neurodata import *
from circuits import *

In [3]:
from qbraid import transpile

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
rescaled_tuning_curves = pd.read_csv(ENV['PWD'] + '/data/rescaled_tuning_curves.csv', index_col=0)
rescaled_tuning_curves

,3.0,4.2,6.0,8.5,12.0,17.0,24.0,33.9,48.0
23,0.272445,0.226280,0.528095,0.342139,0.478668,0.216327,0.388480,0.282726,0.406432
32,0.246955,0.251824,0.261320,0.301364,0.240421,0.677847,0.152107,0.650394,0.359361
35,0.329700,0.345059,0.396044,0.403043,0.472854,0.287826,0.304725,0.239381,0.362960
36,0.293999,0.259318,0.282788,0.360842,0.323867,0.329879,0.353300,0.333539,0.604061
37,0.239892,0.220883,0.447163,0.443715,0.374524,0.325597,0.328626,0.408209,0.352983
...,...,...,...,...,...,...,...,...,...
424,0.211562,0.142474,0.785516,0.386134,0.361296,0.342168,0.184937,0.168489,0.559018
425,0.306663,0.238587,0.426167,0.401231,0.506186,0.294095,0.299420,0.296495,0.372749
426,0.314137,0.290121,0.517580,0.374248,0.256840,0.341100,0.333903,0.349291,0.364372
428,0.307663,0.282272,0.377610,0.382645,0.434304,0.299115,0.426110,0.349712,0.282161


In [5]:
resampled_tuning_curves = pd.read_csv(ENV['PWD'] + '/data/resampled_tuning_curves.csv', index_col=0)
resampled_tuning_curves

,3.0,3.6,4.3,5.2,6.3,7.6,9.1,10.9,13.2,15.8,19.0,22.9,27.6,33.2,39.9,48.0
23,0.049073,0.039918,0.042466,0.075074,0.094048,0.071611,0.064602,0.082698,0.077755,0.044527,0.046081,0.068616,0.063313,0.051262,0.055749,0.073207
32,0.043356,0.043741,0.044285,0.045031,0.046348,0.050228,0.052905,0.044969,0.052667,0.108142,0.099962,0.031631,0.052624,0.111210,0.109811,0.063090
35,0.059220,0.060157,0.062332,0.067351,0.071698,0.071929,0.074087,0.082499,0.080567,0.056549,0.052081,0.055091,0.050466,0.043463,0.047315,0.065194
36,0.054015,0.049772,0.047671,0.049213,0.053367,0.062580,0.065870,0.061018,0.059588,0.060128,0.061826,0.064329,0.064520,0.061341,0.073780,0.110982
37,0.042642,0.037452,0.040609,0.064365,0.080144,0.080781,0.076793,0.069766,0.063875,0.058910,0.057124,0.057708,0.063151,0.071911,0.072028,0.062744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,0.038788,0.022251,0.029875,0.101378,0.141131,0.094036,0.069800,0.067258,0.065521,0.064002,0.056276,0.037474,0.028246,0.029893,0.051580,0.102491
425,0.055056,0.044896,0.043907,0.063648,0.076840,0.072887,0.074830,0.087617,0.085388,0.057789,0.053026,0.053720,0.053528,0.053216,0.056730,0.066920
426,0.056335,0.050662,0.053334,0.078388,0.091691,0.077286,0.062515,0.048800,0.048563,0.059304,0.060755,0.059765,0.060875,0.062461,0.063923,0.065344
428,0.054627,0.050518,0.050672,0.060430,0.067494,0.067470,0.069372,0.075302,0.074053,0.056338,0.058207,0.074734,0.071477,0.062910,0.056297,0.050099


# Transpile circuits

To transpile a pennylane circuit to qiskit, run it under a quantum tape, then use the qbraid transpiler

In [6]:
def circuit_to_tape(pl_circuit):
    def tape_machine(*args):
        with qml.tape.QuantumTape() as tape:
            pl_circuit(*args)
        return tape
    return tape_machine

In [7]:
tape_ang      = circuit_to_tape(circuit_angle_swap )
# circuit_ang_qft  = circuit_to_tape(circuit_angle_qft_swap )
tape_amp      = circuit_to_tape(circuit_amp_iamp       )
tape_amp_qft  = circuit_to_tape(circuit_amp_iamp_qft   )

In [8]:
def small_then_big_array(data1, data2):
    if is_array_lesser(data2, data1):
        return data2, data1
    else:
        return data1, data2

In [9]:
# def get_fidelity_tape(data1:list[int], data2:list[int], tape) -> tuple[Float,Float]:
#     # data1 should be less than data2
#     if is_array_lesser(data2, data1):
#         return tape(data2, data1)
#     else:
#         return tape(data1, data2)

from itertools import combinations


def get_fidelity_circuits(datum, tape_machine):
    quantum_circuits = pd.DataFrame(columns = ['A', 'B', 'circuit'])

    # A x B
    pairs, pairs_len = combinations(datum.index, 2), (len(datum) * (len(datum) - 1) // 2)
    for a, b in tqdm(pairs, total=pairs_len):
        a_i, b_i = datum.loc[a].to_numpy(), datum.loc[b].to_numpy()
        a_i, b_i = small_then_big_array(a_i, b_i)
        tape = tape_machine(a_i, b_i)
        circuit = transpile(tape, "qasm2")
        quantum_circuits.loc[len(quantum_circuits)] = [a, b, circuit]
        # quantum_fidelity.loc[len(quantum_fidelity)] = [b, a, fidelity]

    # A x A
    # for sanity checking
    for a in tqdm(datum.index):
        a_i = datum.loc[a].to_numpy()
        tape = tape_machine(a_i, a_i)
        circuit = transpile(tape, "qasm2")
        quantum_circuits.loc[len(quantum_circuits)] = [a, a, circuit]

    return quantum_circuits


In [10]:
tape_amp(resampled_tuning_curves.loc[23].to_numpy(), resampled_tuning_curves.loc[36].to_numpy())

<QuantumTape: wires=[0, 1, 2, 3], params=4>

In [12]:
ang_circuits = get_fidelity_circuits(rescaled_tuning_curves, tape_ang)
ang_circuits.to_csv(ENV['PWD'] + '/data/circuits_quantum_fidelity_angle_embed.csv')

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

In [13]:
amp_circuits = get_fidelity_circuits(resampled_tuning_curves, tape_amp)
amp_circuits.to_csv(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed.csv')

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

In [14]:
amp_qft_circuits = get_fidelity_circuits(resampled_tuning_curves, tape_amp_qft)
amp_qft_circuits.to_csv(ENV['PWD'] + '/data/circuits_quantum_fidelity_amp_embed_qft.csv')

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

In [ ]:
# Use 2850 shots
# reasoning - that's the number of pairwise comparisons we are making
len(amp_qft_circuits) - 76